In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('../../raw_data/state_sightings.csv', low_memory=False)

In [3]:
data.head()

,datetime,city,state,shape,duration (seconds),state_name,state_population,state_military_bases,state_cities,day,month,month_number,year_number,season,year_season
0,1910-01-01 00:00:00,kirksville (near),mo,disk,120.0,Missouri,7474758.0,2.0,1020.0,1910-01-01,1910-01,1,1910,winter,1910-winter
1,1910-06-01 15:00:00,wills point,tx,cigar,120.0,Texas,33717239.0,5.0,1679.0,1910-06-01,1910-06,6,1910,summer,1910-summer
2,1920-06-11 21:00:00,cicero,in,unknown,60.0,Indiana,6236667.0,1.0,677.0,1920-06-11,1920-06,6,1920,summer,1920-summer
3,1925-12-28 18:00:00,atkinson (6 miles north of),il,disk,60.0,Illinois,18104498.0,1.0,1362.0,1925-12-28,1925-12,12,1925,winter,1925-winter
4,1929-07-05 14:00:00,buchanan (or burns),or,disk,60.0,Oregon,5311863.0,0.0,373.0,1929-07-05,1929-07,7,1929,summer,1929-summer


In [4]:
def value_count_func (series_):
    return len(series_.value_counts())

def most_freq_func (series_):
    x_ = series_.value_counts()
    return list(x_.keys())[0]

def state_year_df (data_, state_):
    
    data_state = data_[data_['state'] == state_]
    
    data_state_season = data_state.groupby(by=['year_number', 'month_number']).agg({'datetime':'count','duration (seconds)':'mean',
                                          'city':value_count_func,'shape':most_freq_func,'day':value_count_func,
                                          'state_population':'first', 'state_military_bases':'first','state_cities':'first'})
    
    data_state_season.rename(columns={'datetime':'sightings_period','duration (seconds)':'avg_duration(seconds)',
                                  'city':'sightings_cities','day':'sightings_days_period'},inplace=True)
    
    data_state_season['state'] = state_
    
    return data_state_season.reset_index().set_index('state')
   

In [5]:
STATES_ = list(data.state.unique()) 

x_ = pd.DataFrame()

for st_ in STATES_:
    
    y_ = state_year_df (data, st_)
    x_ = pd.concat([x_,y_])
    
x_ = x_.reset_index()

x_['year_month'] = x_['year_number'].astype(str)+'-'+x_['month_number'].astype(str)

In [6]:
x_[x_['state'] == 'tx']

,state,year_number,month_number,sightings_period,avg_duration(seconds),sightings_cities,shape,sightings_days_period,state_population,state_military_bases,state_cities,year_month
408,tx,1910,6,1,120.000000,1,cigar,1,33717239.0,5.0,1679.0,1910-6
409,tx,1945,7,1,7200.000000,1,sphere,1,33717239.0,5.0,1679.0,1945-7
410,tx,1946,3,1,20.000000,1,disk,1,33717239.0,5.0,1679.0,1946-3
411,tx,1947,1,1,4.000000,1,other,1,33717239.0,5.0,1679.0,1947-1
412,tx,1948,6,1,10.000000,1,disk,1,33717239.0,5.0,1679.0,1948-6
...,...,...,...,...,...,...,...,...,...,...,...,...
938,tx,2020,12,3,220.000000,3,Oval,3,33717239.0,5.0,1679.0,2020-12
939,tx,2021,1,6,750.000000,5,Circle,5,33717239.0,5.0,1679.0,2021-1
940,tx,2021,2,8,510.000000,7,Oval,6,33717239.0,5.0,1679.0,2021-2
941,tx,2021,3,7,377.142857,6,Light,6,33717239.0,5.0,1679.0,2021-3


In [8]:
x_.to_csv('../../raw_data/features_by_state_year_month.csv',header=True,index=False)